# Lecture 00 : Neural Network 101

written by SinsuSquid (bgkang) on 22 November 2024

## Intro

Project 시작할때는 심술궂은 표정으로 "흥 기초는 안 알려줄건데?"라고 했지만 가오 떨어지잖아요. GNN까지 확장시키기 위해 꼭 필요한 부분들만 간단하게 짚어보도록 합시다.
<br><br>
시작하기에 앞서 꼭 말해주고 넘어가야 하는 fact는, 실은 저도 GNN Model을 개발해서 논문을 submit까지 했지만서도 수업으로 GNN을 배워본적이 없다는 점이에요. 자랑을 하고싶었다기보다는 제가 code를 짜는 방식이 제대로 공부한 사람이 보았을때는 "이자식 왜 이런식으로 했지?" 싶을수도 있다는걸 꼭 말해야 할 것 같았어요.
<br>
같이 따라가는 문제일 것 같은데, 수업을 통해 배운 것이 아니다 보니 제가 하고 있는 설명들이 오개념을 기반으로 만들어졌을수도 있다는 것도 확실히 해야할 것 같아요. 또 한가지 짚어봐야 할 점은 원래 이 분야가 용어 정립이 이제야 막 되가는 단계이기도 하고, 그렇기 때문에 제가 사용하는 단어들이 잘못된 방식으로 사용되고 있을 가능성도 충분히 존재한다는 점입니다. 그래도 일단 이걸로 논문 써서 submit하고 review는 받았으니까 완전 개소리는 아니였다는거 아닐까요? 😏
<br><br>
자, 오늘 배워볼 내용은 기초적인 Neural Network의 개념과, PyTorch를 통해 이를 구현하는 방법이에요. 이를 위해 perceptron의 개념과 Deep Learning (≈ Multi-Layer Perceptron)을 수박 겉 핥기식으로 간단하게만 알아본 다음 PyTorch의 module을 이용해 매우 기초적인 형태의 model까지 만들어보는게 목표랍니다.
<br><br>
이걸 수업으로 진행하면 시간이 얼마나 걸릴까는 생각도 안하면서 안든 자료입니다. 혹여 이 자료를 넘겨줄 일이 생긴다면 잘 생각해보도록 하세요.

## Before you start...

이번 lecture를 따라오기 위한 컴퓨터 환경이 잘 구성되어있는지부터 확인해봐야겠죠. 다음 code를 실행시켰을때 오류는 없는지 확인해보아요.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

위 script에서 오류가 발생했을 때 어떻게 해야할까요?

0. 내가 제대로 된 conda 환경에서 jupyter를 실행한게 맞나 확인해봅니다.
1. (내가 환경을 구성했다면) 필요한 package가 해당 환경에 설치되어있나 확인합니다.
2. (내가 환경을 구성하지 않았다면) 환경을 구성한 사람한테 너는 제대로 하는게 뭐가 있냐고 물어봅니다.

## A Perceptron

새로운 개념이라 놀랄수도 있을 것 같은데, 결국 perceptron도 detail을 다 잘라내고 보면 단순한 function이라는것부터 생각해보고자 합니다.

![Function Box](https://cs.wellesley.edu/~cs115/readings/more-functions-files/images/function.png "cs.wellesley.edu")

여기서 function이라 함은, input과 output이 존재하는데 input이 어떤 과정을 통해서 output을 만들어내는지는 (컴퓨터적인 관점의 function 처럼) 알 필요가 없다는 부분이죠.

Perceptron도 마찬가지입니다. input이 주어졌을 때 output을 만들어낸다는 점은 동일한데, 한가지 아쉬운 점은 우리가 안에서 어떤 일이 일어나는지는 한번은 공부를 대충 해야한다는 거죠 (실제 model 구현하는 code 작성할때는 신경을 좀 덜 써주어도 됩니다. 오늘은 101이니까, 좀만 참아봐요.)

이번엔 perceptron의 내부에서 어떤 일들이 일어나는지를 조금 더 detail하게 표현한 그림을 확인해보도록 하죠.
![A single perceptron](https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Perceptron.svg/1024px-Perceptron.svg.png "en.wikipedia.org")

그림에서 $i_n$, $W_n$은 각각 input, weight vector를 의미합니다. (여담이지만, 여기서의 vector는 physical한 방향을 의미하기보다 Fortran Array처럼 숫자들의 나열을 의미한다고 생각하면 될거에요.) +, f, o는 각각 'summation', 'activation function', 'output'을 의미합니다. 그림에서도 보여주고 있지만, 이처럼 하나의 perceptron에서 정보가 처리되는 과정을 아래 식과 같이 요약할 수 있습니다.

$o = f(\sum^n_{k=1} i_k \cdot W_k)$

우린 이 식에서 의미하는걸 term별로 분리해서 생각해보도록 하죠.

- $i_k \cdot W_k$ : $i_k$와 $W_k$는 vector의 각각 element에 해당하겠죠? 이 둘을 곱한다는건 k-th input ($i_k$)에 가중치 ($W_k$)를 적용하는 과정으로 생각할 수 있어요.
- $\sum^n_{k=1}$ : 위의 단계처럼 가중치를 적용한 input을 모두 더해주는 과정이겠네요. 이 두 단계를 "input을 weighted average"하는 과정이라고 한마디로도 표현할 수 있을거에요.
- $f()$ : $f$는 아무래도 function일 것인데, weighted averaged 된 결과에 activation function ($f$)을 적용하는 단계로 설명합니다. 깊이 설명하진 않을 거지만, 이 activation function이 미분 불가능 (non-differentiable)한 형태를 가진다는 것이 neural network의 핵심입니다. 이러한 function에는 sigmoid, $\tanh$, ReLU 등 여러 종류가 존재하고, 사용자의 필요에 맞게 넣어주면 됩니다.

자, 이쯤 정리를 한번 해볼까요? Perceptron이란 개념은 input을 가중치를 고려하여 summation하고, 이 결과에 activation function을 적용하여 output을 얻는 일종의 function으로 생각할 수 있어요.
<br>
(겁주려는건 아니지만, 이부분이 이해가 안되면 뒷부분 이해를 못하기 때문에 꼭 지금 얘기해주길 바래요.)

## Multi-layered Perceptron (MLP)

누군가를 놀리기 위해 한 문장 적습니다. "우리 다같이 MLP 하네? ^^"


Multi-layer라고 하는거보니 perceptron을 크레이프 케이크마냥 여러 층 겹처놓은 그림을 상상할 수 있겠죠?

![Multi-layered Perceptron](https://media.datacamp.com/legacy/v1725638284/image_bd3b978959.png "datacamp.com")

음, 아무리 봐도 크레이프 케이크는 아닌거같네요.
<br>
그래도 기죽지 말고 시작해보면, 위 그림에서 가운데 'hidden layer (은닉층)' 세로줄 하나가 위에서 봤던 perceptron에 해당한다고 생각하면 될거에요.
<br>
(Bias라는게 하나 추가되었죠? 위에서 input의 weighted average라고 표현하는 부분을 수식으로 $\sum^n_{k=1}(i_k \cdot W_k + b)$ 와 같은 식으로 쓰기도 하는데, 여기서 $b$에 해당하는 term을 bias라고 합니다. 필요에 따라 생략하는 경우도 있어서 위 그림에서는 빠졌던 것 같아요.)

어쨌든! 이 MLP에서 중심이 되는 내용은 앞에서 확인한 perceptron이 여러 층위로 구성되어있다는거겠죠. 다시 말하면, 앞 단계 perceptron의 output이 다음 단계 perceptron의 input으로 사용된다는겁니다!
<br>후에 더 자세하게 설명하겠지만, $W_k$나 $b$라는 값들은 training 과정중에서 변화하게 됩니다 - 이런걸 trainable이라 표현해요. Perceptron을 여러 층으로 쌓는다는건 trainable parameter의 갯수를 효율적으로 늘려준다는걸 이야기하겠죠.

그런데 여기서 한가지 궁금한게 나올법해요. "아니 Neural Network (NN)을 소개해준다 하면서 perceptron 얘기만 주구장창 하고 있잖아요!" 그것은 왜냐하면, NN이란 사실 많은 층위의 perceptron과 같은 의미이기 때문이에요! 다시말해, "Very-many-layered Perceptron이 Neural Network다!"라고도 이야기할 수 있다는 겁니다. 

## So, what's "Training"?

그렇다면 우리가 흔히 "모델을 training한다"라고 표현을 많이 하는데, "학습"이라는 단어가 의미하는건 무엇일까요?

이를 생각해보기 위해 우리가 인공지능에게 기대하는 것이 무엇인지부터 다시 출발해보도록 해요.
<br>
인공지능한테는 미안한 얘기지만, 결국 인공지능도 하나의 function이라 과감하게 표현해볼게요. 그 말은 즉슨, 어떠한 input을 제공하였을 때 알잘딱깔센으로 input을 잘 처리하여 우리가 원하는 정확한 output을 얻어내는걸 인공지능에게 기대한다 이말입니다. 지도 학습 (supervised learning)이란 결국 더럽게 많은 \[input, (desired) output\] set을 model이 읽어보도록 한 후 model이 "아 이런 input을 받았을 때 이런 output을 주길 원하는구나!"를 깨우치게끔 유도하는 과정이라고 소개하겠습니다.

그렇다면 이번엔 model은 어떤 과정을 통해 이를 "깨우치게" 되는가 이야기해보죠. 앞에서 "trainable" parameter에 해당하는것이 weight ($W_k$)와 bias ($b$)라고 표현했는데, "trainable"이 의미하는 것은 학습 과정중에서 이 값이 계속해서 변한다는걸 의미해요. 앞뒤를 바꿔서 말해볼까요? Model이 train하는 과정은 weight와 bias를 계속 변화시켜가면서 주어진 input으로부터 원하는 output이 나오도록 유도하는 과정이 되겠죠. Weight와 bias를 변화시키면 위에서 보았던 $o = f(\sum^N_{k=1}(i_k \cdot W_k + b))$는 $W_k$와 $b$ 값의 조합에 따라 매번 다른 output을 만들어낼거에요. 우리가 궁극적으로 원하는건 desired output을 생성할 수 있는 특정한 $W_k$와 $b$의 조합을 찾는게 될 것이고, 이 과정을 training이라 할 수 있습니다. 물론, 효과적인 학습을 위해서는 weight와 bias에 계속해서 random 값을 넣어가며 output을 확인하는것보다는 좋은 algorithm을 필요로 할거에요.

그 "좋은 algorithm"에 대해서도 기꺼이 이야기 해야겠지만 이번 lecture에서는 과감하게 오차역전파 (back propagation) 및 경사하강법 (gradient descent)에 대한 설명을 빼볼까 해요. 그렇다고 이 개념을 몰라도 된다고 생각하면 큰 오산입니다. 실제 coding을 하면서는 "아 optimizer로 `torch.optim.Adam` 쓰고 `lr = 1E-3`으로 설정했구나" 하면 끝날 일이기도 하고, YouTube나 Google에 검색하면 저보다도 훨씬 설명을 더 잘해주시는 분들이 계시기 때문이죠. 우리의 목표는 Model 생성을 위한 기초적인 내용만 햝고 가는것이기에 지금은 이런 "좋은 algorithm"을 optimizer라 부르는구나 정도 생각하고 꼭 나중에 공부해보도록 해요.

## Close the book, we're coding.

Tutorial식으로 구성해보려 했는데 code는 하나도 없고 무슨 책이라도 쓰고 있는 기분이네요. 걱정 말아요. 이제 coding좀 해봅시다. 우리의 이번 lecture 목표는 multi-layer perceptron을 PyTorch로 구현하는겁니다. (원래 training/test cycle도 같이 할까 생각은 해봤는데 양이 많아질 것 같기도 하고 더 큰 문제는 제가 피곤하네요.)

처음으로 code를 lecture에 집어넣다보니 설명을 좀 하고 시작할게요.

- code를 보며 하고싶은 말은 주석 (comment - # 으로 표시)으로 표현할거에요. code를 읽어보면서 꼭꼭 comment도 잘 읽어보기로 해요.
- comment에만 써있고 본문에 없는 내용들은 "웅앵 내가 이런것까지 알려주기 귀찮아 너가 알아와" 라는 뜻이에요.

자, 그러면 PyTorch에서 percepron은 어떻게 표현할까요?

In [ ]:
import torch

perceptron = torch.nn.Sequential(
    torch.nn.Linear(in_features = 10,
                    out_features = 2,
                    bias = True,
                    device = None,
                    dtype = torch.float64),
    torch.nn.ReLU()
)

perceptron

입 아프게 설명한게 꼴랑 code 몇줄이면 된다는게 마음이 조금 아프네요? 그만큼 PyTorch가 위대하다는걸 강조할 수 있는 기회가 되겠네요.

PyTorch를 처음 써보는거니 조금 detail하게 설명을 해볼게요.

1. `torch.nn.Sequential` : container라고 부르는데, sequential이 해주는 역할은 container안에 있는 layer (`torch.nn.Linear`, `torch.nn.ReLU` 같은놈들)을 차례로 적용시키겠다~ 이말입니다.
2. `torch.nn.Linear` : Perceptron 수식에서 (자주 써먹네요) $\sum^n_{k=1}(i_k \cdot w_k + b)$ 까지 해당하는 부분입니다. 사실 `bias`, `device`, `dtype`의 경우는 꼭 지정하지 않아도 기본값이 알아서 들어가요. 다시말해 `in_features`, `out_features` : 입/출력값의 갯수만 알려주면 된다는거죠. 참 쉽죠?
3. `torch.nn.ReLU` : 수식에서는 $f()$에 해당하는 부분이고, activation function이라고 불렀죠. PyTorch는 여러 종류의 activation function을 제공하고 있는데, 그 중 ReLU함수를 써봤습니다.

마지막에 `perceptron`이라는 object의 `__str__()` method를 호출해봤는데요, Sequential이라는 container에 포함된 layer의 정보가 출력되는걸 확인할 수 있네요.

이번엔 앞에서 봤던 MLP 그림을 PyTorch로 구현해볼까요?

In [ ]:
import torch

# 눈치챘는지 모르겠지만, 저는 좀 변태같이 alias를 잘 안쓰는 편이에요.

class Perceptron(torch.nn.Module): # We call it "Pythonic".
    def __init__(self, in_features: int, out_features: int):
        self.in_features = in_features
        self.out_features = out_features
        
        super().__init__()
        self.linear = torch.nn.Linear(in_features = self.in_features, 
                                      out_features = self.out_features)
        self.relu = torch.nn.ReLU()

        return

    def forward(self, x):
        x = self.linear(x)
        return self.relu(x)

    def reset_parameters(self):
        self.linear.reset_parameters()

multi_layer = torch.nn.Sequential(
    Perceptron(4, 4),
    Perceptron(4, 4)
)

multi_layer

이번엔 변화구를 좀 넣어봤죠? Project 시작하기 전에 겁은 충분히 줬다고 생각하니까, `class` 부분은 알아서 공부해왔으면 좋겠어요.

이번에도 실질적으로 MLP를 구성하는 code는
```python
multi_layer = torch.nn.Sequential(
    Perceptron(4, 4),
    Perceptron(4, 2)
)
```
가 전부였죠.
<br>왜 PyTorch를 써야하는지 느껴본다면 좋겠어요.

이번엔 당장은 별 의미가 없긴 하지만 실제로 MLP를 통헤 값이 출력되는걸 한번 보도록 할게요.

In [ ]:
dummy_in = torch.tensor([1.0, 2.0, 3.0, 4.0])
# torch.tensor : numpy 의 ndarray랑 비슷하다고 생각하면 됩니다.
# 오차 역전파에 필요한 기울기 (gradient) 계산이 자동으로 수행되는게 특징이에요.
# 값 계산 + 기울기 계산도 같이 하니까 numpy.ndarray보단 속도가 좀 느리겠죠?

dummy_out = multi_layer(dummy_in) # 사용은 function처럼 하면 돼요.
print("Before : ", dummy_out)

for layer in multi_layer: # weight를 다시 설정하니,
    layer.reset_parameters()

dummy_out = multi_layer(dummy_in) # 출력값이 바뀌었네요?
print("After : ", dummy_out)

이번엔 output도 한번 살펴보죠.
<br>
`torch.nn.Sequential`의 출력값은 일단 `torch.tensor`인거같네요.
`torch.tensor`에는 `grad_fn`이란 property를 갖고있네요.
`ReluBackward0`라는건 아마 activation function으로 `ReLU`를 사용했기 때문이겠죠?

한가지 더,
perceptron에 있는 layer의 trainable parameters (weight, bias)를 `.reset_parameters()` method를 통해 바꿔주니까 출력값도 바뀌는걸 확인할 수 있었어요!

## Wrap Up!

마지막에 code를 우다다다 적어놓고 제대로 설명도 안하는데는 이유가 있겠죠? 시작하기 전에 충분히 경고는 줬으니까, code가 어떻게 굴러가는지와 내가 code에 대해 설명한 부분을 이해할 수 있도록 준비를 해준다면 좋겠어요. 다시 말하지만, 들꽃반에서 `Python`까지 가르쳐줄 생각은 없습니다

조금 마음 아픈 얘기긴 하지만, 사실 오늘 Lecture에 내용은 Perceptron, Multi-Layer Perceptron 설명에 Machine Learning에서 "training"의 의미를 살짝 소개한게 다에요.
<br>
PyTorch로 code 짜는거? code 길이 보면 알겠지만 쉬운 일이고 누구나 할 수 있어요. 우린 그것보단 더 잘 해야되니까, 앞으로 갈 길이 많다는걸 얘기해보며 이번 시간을 마무리해보려 해요.